## ACL Quickstart for Azure AI Search

## 1. Load Connections

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "acl-sample")
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")


## 2. Create Sample Index

In [2]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

Index 'acl-sample' created with permission filter option enabled.


## 3. Connect to Graph to find your oid and groups

In [3]:
from msgraph import GraphServiceClient
client = GraphServiceClient(credentials=credential, scopes=["https://graph.microsoft.com/.default"])

groups = await client.me.member_of.get()
me = await client.me.get()
oid = me.id

## 4. Upload Sample Data

In [14]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

documents = [
    { "id": "1", "oid": [oid], "group": [groups.value[0].id], "name": "Document 1" },
    { "id": "2", "oid": ["all"], "group": [groups.value[0].id], "name": "Document 2" },
    { "id": "3", "oid": [oid], "group": ["all"], "name": "Document 3" },
    { "id": "4", "oid": ["none"], "group": ["none"], "name": "Document 4" },
    { "id": "5", "oid": ["none"], "group": [groups.value[0].id], "name": "Document 5" },
]
search_client.upload_documents(documents=documents)
print("Documents uploaded to the index.")


Documents uploaded to the index.


## 5. Search sample data with x-ms-query-source-authorization

In [16]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

Name: Document 1, OID: ['44e5f686-35a1-4e2c-9ff3-7b6adafcd3c3'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']
Name: Document 2, OID: ['all'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']
Name: Document 3, OID: ['44e5f686-35a1-4e2c-9ff3-7b6adafcd3c3'], Group: ['all']
Name: Document 5, OID: ['none'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']


## 6. Search sample data without x-ms-query-source-authorization

In [17]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

Name: Document 1, OID: ['44e5f686-35a1-4e2c-9ff3-7b6adafcd3c3'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']
Name: Document 2, OID: ['all'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']
Name: Document 3, OID: ['44e5f686-35a1-4e2c-9ff3-7b6adafcd3c3'], Group: ['all']
Name: Document 4, OID: ['none'], Group: ['none']
Name: Document 5, OID: ['none'], Group: ['ec5aece9-33fc-4b2e-abe1-aedf771357a3']
